# Task 6.2 from *SemEval*
## Imports

In [1]:
from src.dataprocessing import preprocesser, postprocesser
from src.model import model
from src.thresholdadjuster import find_thresholds

import torch
import json
import numpy as np

## Input Data Preprocessing

In [2]:
# preprocesser.augment_data("propaganda_detection/datasets/train-articles","propaganda_detection/datasets/train-labels-task2-technique-classification", "out/augmented_data")

In [3]:
train_file_path = "SEMEVAL-2021-task6-corpus/data/training_set_task2.txt"
with open(train_file_path, "r") as file:
    train_data = json.load(file)

train_aug_file_path = "out/augmented_data.json"
with open(train_aug_file_path, "r") as file:
    train_aug_data = json.load(file)

dev_file_path = "SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt"
with open(dev_file_path, "r") as file:
    dev_data = json.load(file)

test_file_path = "SEMEVAL-2021-task6-corpus/data/test_set_task2.txt"
with open(test_file_path, "r") as file:
    test_data = json.load(file)

input_tdata, output_tdata = preprocesser.preprocess_data(train_data)
input_taugdata, output_taugdata = preprocesser.preprocess_data(train_aug_data)
input_vdata, output_vdata = preprocesser.preprocess_data(dev_data)

# Data Augmentation ö
# input_tdata.extend(input_taugdata)
# output_tdata.extend(output_taugdata)

In [4]:
model.train(input_tdata, output_tdata, input_vdata, output_vdata, model_name='roberta-large')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be ab

KeyboardInterrupt: 

## Results & Post-Processing

In [4]:
model_roberta = torch.load('models/roberta-large-MEMbErt-bce.pth')
model_robetwitter = torch.load('models/roberta-twitter-MEMbErt-bce.pth')
bertweet = torch.load('models/bertweet-MEMbErt-bce.pth')

predictions = []

with torch.no_grad():
    for meme in input_vdata:
        model_roberta.eval()
        model_robetwitter.eval()
        bertweet.eval()

        res_roberta = model_roberta(' '.join(meme)).cpu().numpy()
        res_robetwitter = model_robetwitter(' '.join(meme)).cpu().numpy()
        res_bertweet = bertweet(' '.join(meme)).cpu().numpy()

        predictions.append(np.power(res_roberta * res_robetwitter * res_bertweet, (1/3)))
        # predictions.append(0.15*res_roberta + 0.65*res_robetwitter + 0.2*res_bertweet)

In [5]:
best_thresholds = find_thresholds.get_best_thresholds(dev_data, input_vdata, predictions, model_robetwitter.tokenizer)

Getting best thresholds...: 100%|███████████████████████████████████████████████████████████████████████████████████| 101/101 [00:40<00:00,  2.52it/s]


In [10]:
best_thresholds[0]

0.14

In [7]:
output = postprocesser.postprocess_data(dev_data, input_vdata, predictions, model_robetwitter.tokenizer, threshold=best_thresholds)
with open("out/out-dev.txt", "w") as f:
    json.dump(output, f)

In [9]:
input_test_data, _ = preprocesser.preprocess_data(test_data)

model_roberta = torch.load('models/roberta-large-MEMbErt-bce.pth')
model_robetwitter = torch.load('models/roberta-twitter-MEMbErt-bce.pth')
bertweet = torch.load('models/bertweet-MEMbErt-bce.pth')

predictions = []

with torch.no_grad():
    for meme in input_test_data:
        model_roberta.eval()
        model_robetwitter.eval()
        bertweet.eval()

        res_roberta = model_roberta(' '.join(meme)).cpu().numpy()
        res_robetwitter = model_robetwitter(' '.join(meme)).cpu().numpy()
        res_bertweet = bertweet(' '.join(meme)).cpu().numpy()

        predictions.append(np.power(res_roberta * res_robetwitter * res_bertweet, (1/3)))
        # predictions.append(0.15*res_roberta + 0.65*res_robetwitter + 0.2*res_bertweet)

output = postprocesser.postprocess_data(test_data, input_test_data, predictions, model_robetwitter.tokenizer, threshold=best_thresholds)
with open("out/out.txt", "w") as f:
    json.dump(output, f)

## Evaluation
For dev evaluation, type:

`python3 SEMEVAL-2021-task6-corpus/scorer/task2/task-2-semeval21_scorer.py -s out/out-dev.txt -r SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt -p SEMEVAL-2021-task6-corpus/techniques_list_task1-2.txt
`

For test evaluation, type:
`python3 SEMEVAL-2021-task6-corpus/scorer/task2/task-2-semeval21_scorer.py -s out/out.txt -r SEMEVAL-2021-task6-corpus/data/test_set_task2.txt -p SEMEVAL-2021-task6-corpus/techniques_list_task1-2.txt
`